In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
tond = pd.read_csv("type_before_pre_ton.csv")

In [ ]:
iot20 = pd.read_csv("iot2020.csv")

In [ ]:
#5:5 비율을 맞추기 위해 ton-iot의 개수만큼 랜덤 추출
iot20 = iot20.sample(n=461043, replace=False, axis=0)

In [ ]:
del tond['Unnamed: 0'] 

In [ ]:
tond['type'].value_counts()

In [ ]:
#feature명 통일
tond.rename(columns={'type':'Cat'})

In [ ]:
#공격명 통일
iot20 = iot20.replace('Normal', 'normal' )
iot20 = iot20.replace('DoS', 'dos' )
iot20 = iot20.replace('MITM ARP Spoofing', 'mitm' )
iot20 = iot20.replace('Scan', 'scanning' )

In [ ]:
iot20 = pd.get_dummies(iotd, columns=['conn_state'])  

In [ ]:
iot20 = iot20.drop(columns=[ 'Label','Sub_Cat','Flow_ID', 'Src_IP', 'Dst_IP',  'Timestamp', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port'])

In [ ]:
tond = tond.drop(columns=[ 'ts', 'src_port', 'dst_port'])

In [ ]:
#feature명 통일
tond= tond.rename(columns={'duration':'Flow_Duration','src_pkts':'Tot_Fwd_Pkts', 'dst_pkts':'Tot_Bwd_Pkts', 'proto':'Protocol', 'src_bytes':'Fwd_Pkt_Len_Mean' , 'dst_bytes':'Bwd_Pkt_Len_Mean', 'src_ip_bytes':'Fwd_Pkts/s', 'dst_ip_bytes':'Bwd_Pkts/s', 'type':'Cat', 'ts':'Timestamp', 'src_port':'Src_Port', 'dst_port':'Dst_Port', 'label':'Label'})

In [ ]:
df_merge = df_merge.drop(columns=['Label'])

In [ ]:
#데이터 합치기
frames = [iot20, tond]
df_merge = pd.concat(frames)

In [ ]:
#결측값 존재 여부 확인
df_merge.isna().sum()

In [ ]:
#결측값 채우기
tond.fillna(-1,inplace=True)
iot20.fillna(-1,inplace=True)
df_merge.fillna(-1,inplace=True)

In [ ]:
iot20['Init_Bwd_Win_Byts']=np.NaN
iot20['Flow_Pkts/s']=np.NaN
iot20['Flow_IAT_Max']=np.NaN
iot20['Bwd_IAT_Tot']=np.NaN

ton['conn_state']=np.NaN
ton['service']=np.NaN

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [ ]:
X = df_merge[['Protocol', 'Flow_Duration', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Mean', 'Tot_Fwd_Pkts', 'Fwd_Pkts/s', 'Tot_Bwd_Pkts', 'Bwd_Pkts/s', 'conn_state', 'Init_Bwd_Win_Byts', 'Flow_Pkts/s', 'Bwd_IAT_Tot', 'service', 'Flow_IAT_Max']]
Y = df_merge['Cat']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
Z = iot20[['Protocol', 'Flow_Duration', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Mean', 'Tot_Fwd_Pkts', 'Fwd_Pkts/s', 'Tot_Bwd_Pkts', 'Bwd_Pkts/s', 'conn_state', 'Init_Bwd_Win_Byts', 'Flow_Pkts/s', 'Bwd_IAT_Tot', 'service', 'Flow_IAT_Max']]
W = iot20['Cat']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()
print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))